In [38]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
!pip install transformers

In [40]:
from transformers import BertForSequenceClassification, BertTokenizer, BertConfig,AdamW , get_linear_schedule_with_warmup
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import itertools
import pandas as pd
import numpy as np
import random

In [41]:
random.seed(42)

In [42]:
cd /content/drive/MyDrive/master_thesis/inputs/

/content/drive/MyDrive/master_thesis/inputs


In [43]:
ls

ENCFF292FVY.tsv  ENCFF910TAZ.tsv  hg38_msxTm_ENCFF292FVY/  hg38_msxTm.txt
ENCFF694PZC.tsv  find_motifs/     hg38_msxTm_ENCFF910TAZ/


In [44]:
input_csv = "hg38_msxTm_ENCFF910TAZ/hg38_msxTm_ENCFF910TAZ.csv"

Import input that was parsed using inputparser.ipynb

In [45]:
final_matrix = pd.read_csv(input_csv)

choose pretrained model from 3 to 6 mer (!!!change seq2kmer k)

In [46]:
cd /content/drive/MyDrive/master_thesis/models/4-new-12w-0/

/content/drive/MyDrive/master_thesis/models/4-new-12w-0


In [47]:
import torch
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print("Using GPU.")
else:
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")

Using GPU.


Parse sample data into pytorch object to pass through BERT trainer https://huggingface.co/transformers/custom_datasets.html https://huggingface.co/transformers/main_classes/trainer.html#transformers.Trainer
https://huggingface.co/transformers/custom_datasets.html

In [11]:
DNABERT_Tokenizer = BertTokenizer(vocab_file = "vocab.txt", config = "tokenizer_config.json")

In [12]:
def seq2kmer(seq, k = 4):
    """
    Function provided by Ji et al. (https://github.com/jerryji1993/DNABERT)
    Will convert given sequence into kmer.
    """
    kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
    kmers = " ".join(kmer)
    return kmer

In [13]:
def tokenize(seq_kmers, labels):
  input_ids = []
  attention_masks = []

  for sent in seq_kmers:
      encoded_dict = DNABERT_Tokenizer.encode_plus(
                          sent,                      # Sentence to encode.
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                          # Pad & truncate all sentences. (does not apply for our data since the lengths are all the same)
                          max_length = 297,
                          pad_to_max_length = True,
                          return_attention_mask = True,   # Construct attention masks.
                          return_tensors = 'pt',     # Return pytorch tensors.
                    )
      
      # Add the encoded sentence to the list.    
      input_ids.append(encoded_dict['input_ids'])
      
      # And its attention mask (simply differentiates padding from non-padding).
      attention_masks.append(encoded_dict['attention_mask'])

  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  labels = torch.tensor(labels)

  # Combine the training inputs into a TensorDataset.
  dataset = TensorDataset(input_ids, attention_masks, labels)
  return dataset

In [14]:
seq_kmers = list(map(seq2kmer, final_matrix["Sequence"].tolist()))
dataset = tokenize(seq_kmers, final_matrix["logTPM"].tolist())

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [16]:
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=1)
train_dataset, val_dataset = train_test_split(train_dataset, test_size=0.25, random_state=1)

In [17]:
print("dataset size = ", len(dataset))
print("train_dataset size = ", len(train_dataset))
print("val_dataset size = ", len(val_dataset))
print("test_dataset size = ", len(test_dataset))


dataset size =  1919
train_dataset size =  1151
val_dataset size =  384
test_dataset size =  384


In [19]:
model_config_path = "./config.json"
model_config =BertConfig.from_pretrained(model_config_path, num_labels = 1)
model_weight_path = "./pytorch_model.bin"
dnabert = BertForSequenceClassification.from_pretrained(model_weight_path, config = model_config)

Some weights of the model checkpoint at ./pytorch_model.bin were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./pytorch_model.bin and a

In [20]:
# Get all of the model's parameters as a list of tuples.
params = list(dnabert.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                    (261, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

BERT authors suggest:
batch sizes: 8, 16, 32, 64, 128
learning rates: 3e-4, 1e-4, 5e-5, 3e-5

https://github.com/google-research/bert

In [ ]:
batch_size = 16

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

batch_lr = {8:3e-4, 16:1e-4, 32: 5e-5, 64: 3e-5, 128: 3e-5}

optimizer = AdamW(dnabert.parameters(), lr=batch_lr[batch_size])

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 4

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

model = dnabert
model.to(device)                                  

In [ ]:
def train(model, optimizer, scheduler, epochs,       
          train_dataloader, validation_dataloader, device, clip_value=2):
    training_stats = []

    for epoch_i in range(0, epochs):
        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')
        total_train_loss = 0
        for batch in train_dataloader:
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            labels = batch[2].to(device)
            #https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch
            
            optimizer.zero_grad()
            model.zero_grad()

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs[0]
            total_train_loss += loss.item()
            loss.backward()
            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
            optimizer.step()
            scheduler.step()

        avg_train_loss = total_train_loss / len(train_dataloader)
        print("  Average training loss: {0:.2f}".format(avg_train_loss))

        # ========================================
        #               Validation
        # ========================================
        # After the completion of each training epoch, measure our performance on
        # our validation set.
        
        print("")
        print("Running Validation...")

        model.eval()

        # Tracking variables 
        total_eval_accuracy = 0
        total_eval_loss = 0
        nb_eval_steps = 0

        # Evaluate data for one epoch
        for batch in validation_dataloader:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            
            # Tell pytorch not to bother with constructing the compute graph during
            # the forward pass, since this is only needed for backprop (training).
            with torch.no_grad():
                output = model(b_input_ids, 
                                      token_type_ids=None, 
                                      attention_mask=b_input_mask,
                                      labels=b_labels)
            
            loss = output[0]
            # Accumulate the validation loss.
            total_eval_loss += loss.item()


        # Calculate the average loss over all of the batches.
        avg_val_loss = total_eval_loss / len(validation_dataloader)
        
        print("  Validation Loss: {0:.2f}".format(avg_val_loss))

        # Record all statistics from this epoch.
        training_stats.append(
            {
                'epoch': epoch_i + 1,
                'Training Loss': avg_train_loss,
                'Valid. Loss': avg_val_loss,
            }
        )
    return model, training_stats

In [ ]:
ft_dnabert_model, training_stats = train(model, optimizer, scheduler, epochs, 
              train_dataloader, validation_dataloader, device, clip_value=2)

In [ ]:
import pandas as pd

# Display floats with two decimal places.
pd.set_option('precision', 2)

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')

# Display the table.
df_stats

In [ ]:
import matplotlib.pyplot as plt
% matplotlib inline

import seaborn as sns

# Use plot styling from seaborn.
sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")

# Label the plot.
plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks(list(range(1, 5)))

plt.show()

In [22]:
cd /content/drive/MyDrive/master_thesis/models/4-new-12w-0/ft_dnabert_model_4mer_save/

/content/drive/MyDrive/master_thesis/models/4-new-12w-0/ft_dnabert_model_4mer_save


In [23]:
DNABERT_Tokenizer = BertTokenizer(vocab_file = "vocab.txt", config = "tokenizer_config.json")

ft_model_config_path = "./config.json"
ft_model_config =BertConfig.from_pretrained(ft_model_config_path, num_labels = 1)

ft_model_weight_path = "./pytorch_model.bin"
ft_dnabert_model = BertForSequenceClassification.from_pretrained(ft_model_weight_path, config = ft_model_config)

Save the best performing model (4mer) in output dir

In [ ]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = './ft_dnabert_model_4mer_save/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(ft_dnabert_model, 'module') else ft_dnabert_model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
DNABERT_Tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
#torch.save(args, os.path.join(output_dir, 'training_args.bin'))

Make prediction on test data

In [24]:
def predict(model, test_dataset, device):

  batch_size = 1
  prediction_dataloader = DataLoader(test_dataset, batch_size, shuffle=True)
  # Put model in evaluation mode
  model.eval()
  # Tracking variables
  max_seq_length = 297 #this variable is subject to change when kmer model and sequence length varies
  preds = None
  true_labels = []
  attention_scores = np.zeros([len(test_dataset), 12, max_seq_length, max_seq_length])
  sequences = []
  model.to(device)
  # ========================================
  #               Prediction
  # ========================================
  print("")
  print("Running Prediction...")

  for index, batch in enumerate(prediction_dataloader):
      b_input_ids = batch[0].to(device)
      b_input_mask = batch[1].to(device)
      b_labels = batch[2].to(device)

      # Telling the model not to compute or store gradients, saving memory and 
      # speeding up prediction
      with torch.no_grad():
          # Forward pass, calculate logit predictions
          #b_label not required for predictions, no loss is calucalated
          outputs = model(b_input_ids, token_type_ids=None, 
                          attention_mask=b_input_mask, output_attentions=True)
          """
          attention = outputs[-1][-1]
          _, logits = outputs[:2]

          preds[index*batch_size:index*batch_size+len(batch[0]),:] = logits.detach().cpu().numpy()
          attention_scores[index*batch_size:index*batch_size+len(batch[0]),:,:,:] = attention.cpu().numpy()
                # if preds is None:
                #     preds = logits.detach().cpu().numpy()
                # else:
                #     preds = np.concatenate((preds, logits.detach().cpu().numpy()), axis=0)

                # if attention_scores is not None:
                #     attention_scores = np.concatenate((attention_scores, attention.cpu().numpy()), 0)
                # else:
                #     attention_scores = attention.cpu().numpy()
          """
          logits = outputs[0]
          if preds is None:
              preds = logits.detach().cpu().numpy()
          else:
              preds = np.concatenate((preds, logits.detach().cpu().numpy()), axis=0)
          true_labels.append(b_labels)
          attention = outputs[-1][-1].cpu().numpy()
          attention_scores[index*batch_size:index*batch_size+len(batch[0]),:,:,:] = attention
          sequences.append(b_input_ids.cpu().tolist())
  
  sequences=np.array(sequences).squeeze()


  print("")
  print("Prediction completed")

  return preds, true_labels, attention_scores, sequences


In [25]:
torch.cuda.empty_cache()

In [ ]:
predictions, labels, attention_scores, sequences = predict(ft_dnabert_model, test_dataset, device)


Running Prediction...

Prediction completed


In [52]:
cd /content/drive/My Drive/master_thesis/inputs/find_motifs/

/content/drive/My Drive/master_thesis/inputs/find_motifs


In [34]:
#removed probs because dont know what it does yet

def unknown_codes(attention_scores, kmer = 4):

    """
    Seems like it reduced dimension from (batchsize, heads, kmer-len, kmer-len) 
    to (batchsize, kmer-len)
    """
    scores = np.zeros([attention_scores.shape[0], attention_scores.shape[-1]])

    for index, attention_score in enumerate(attention_scores):
        attn_score = []
        for i in range(1, attention_score.shape[-1]-kmer+2):
            attn_score.append(float(attention_score[:,0,i].sum()))

        for i in range(len(attn_score)-1):
            if attn_score[i+1] == 0:
                attn_score[i] = 0
                break

        # attn_score[0] = 0    
        counts = np.zeros([len(attn_score)+kmer-1])
        real_scores = np.zeros([len(attn_score)+kmer-1])
        for i, score in enumerate(attn_score):
            for j in range(kmer):
                counts[i+j] += 1.0
                real_scores[i+j] += score
        real_scores = real_scores / counts
        real_scores = real_scores / np.linalg.norm(real_scores)

        scores[index] = real_scores
        

    return scores

In [65]:
def kmer2seq(kmers):
    """
    Convert kmers to original sequence
    
    Arguments:
    kmers -- str, kmers separated by space.
    
    Returns:
    seq -- str, original sequence.
    """
    kmers_list = kmers.split(" ")
    bases = [kmer[0] for kmer in kmers_list[0:-1]]
    bases.append(kmers_list[-1])
    seq = "".join(bases)
    assert len(seq) == len(kmers_list) + len(kmers_list[0]) - 1
    return seq

In [66]:
def decode(tokenized_seqs):
    seqs = None
    for tokenized_seq in tokenized_seqs:
        seq = DNABERT_Tokenizer.decode(tokenized_seq,
                                        skip_special_tokens = True
                                        )
        seq = kmer2seq(seq)
        
        if seqs is None:
            seqs = seq
        else:
            seqs = np.hstack((seqs, seq))
    return seqs

In [53]:
sequences = np.load("sequences.npy")

In [67]:
sequences_2 = decode(sequences)

In [68]:
print(sequences_2[0])

GGAGGAGTTGCACAGTCAATGCCTTGTTCAAGATTAATCATTTTCCCTGTTGCCTATGACTGAGGCTGTTTCTCACATCCTGCCCCTGGTTTTGCCTGTCCCAAGGATTTCGTCTGAAGGGCGGGACATTCCCCCTGCCTCTTCGCACCACAGCCAGAGCCTGCCATTAGGACCAATGAAAGCAAAGTACCTCATCCCCTCAGTGACTAAGAATCGCAGTATTTAAGAGGTAGCAGGAATGGGCTGAGAGTGGTGTTTGCTTTCTCCACCAGAAGGGCACACTTTCATCTAATTTGGG


In [69]:
att_2 = unknown_codes(att)
np.save("positives_seq.npy", sequences_2) #save positive sequences in seq format
np.save("attention_scores_2d.npy", att_2) #save attention_scores for positive sequences in (batchsize, no.kmer)

Interpret prediction vs true label

https://stats.stackexchange.com/questions/131267/how-to-interpret-error-measures

In [ ]:
def unlist(predictions, true_labels):
    predictions = (list(itertools.chain(*predictions)))
    predictions = (list(itertools.chain(*predictions)))
    true_labels = [label.tolist() for label in true_labels]
    true_labels = (list(itertools.chain(*true_labels)))
    return predictions, true_labels

def measurement_metric(predictions, true_labels, measurement):
    metric = measurement(true_labels, predictions)               
    return metric

In [ ]:
pred, lab = unlist(predictions, labels)
mse = measurement_metric(pred, lab, mean_squared_error)
r2 = measurement_metric(pred, lab, r2_score)
print(mse, r2)

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(lab, pred, c='crimson')
#plt.yscale('log')
#plt.xscale('log')

p1 = max(max(pred), max(lab))
p2 = min(min(pred), min(lab))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.xlabel('True Values', fontsize=15)
plt.ylabel('Predictions', fontsize=15)
plt.axis('equal')
plt.show()